In [228]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [229]:
filepath = '.\input\AMZN_2006-01-01_to_2018-01-01.csv' # CHANGE THE FILE PATH FOR WHICH YOU WANT TO START PREDICTING
data = pd.read_csv(filepath)
data = data.sort_values('Date')
# data.head(10)

In [230]:
data.shape[1]

7

In [231]:

import matplotlib.pyplot as plt
import seaborn as sns
"""
sns.set_style("darkgrid")
plt.figure(figsize = (15,9))
plt.plot(data[['Close']])
plt.xticks(range(0,data.shape[0],500),data['Date'].loc[::500],rotation=45)
plt.title("YOUR Stock Price",fontsize=18, fontweight='bold')
plt.xlabel('Date',fontsize=18)
plt.ylabel('Close Price (USD)',fontsize=18)
plt.show()
"""


'\nsns.set_style("darkgrid")\nplt.figure(figsize = (15,9))\nplt.plot(data[[\'Close\']])\nplt.xticks(range(0,data.shape[0],500),data[\'Date\'].loc[::500],rotation=45)\nplt.title("YOUR Stock Price",fontsize=18, fontweight=\'bold\')\nplt.xlabel(\'Date\',fontsize=18)\nplt.ylabel(\'Close Price (USD)\',fontsize=18)\nplt.show()\n'

In [232]:

price = data[['Close']]
price.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3019 entries, 0 to 3018
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   3019 non-null   float64
dtypes: float64(1)
memory usage: 47.2 KB


In [233]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1, 1))
price['Close'] = scaler.fit_transform(price['Close'].values.reshape(-1,1))

C:\Users\Frank\AppData\Local\Temp\ipykernel_16404\68737012.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [234]:
def split_data(stock, lookback): # will need to be modified to fit project
    data_raw = stock.to_numpy() # convert to numpy array
    data = []
    
    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - lookback): 
        data.append(data_raw[index: index + lookback])
    
    data = np.array(data)
    # 20% data.shape[0]
    test_set_size = data.shape[0]
    
    x_test = data[:test_set_size,:-1]
    y_test = data[:test_set_size,-1,:]
    
    # return [x_train, y_train, x_test, y_test]
    return [x_test, y_test]

In [235]:
lookback = 20 # choose sequence length
x_test, y_test = split_data(price, lookback)
print('x_test.shape = ',x_test.shape)
print('y_test.shape = ',y_test.shape)

x_test.shape =  (2999, 19, 1)
y_test.shape =  (2999, 1)


In [236]:
import torch
import torch.nn as nn

x_test = torch.from_numpy(x_test).type(torch.Tensor)
y_test_lstm = torch.from_numpy(y_test).type(torch.Tensor)
y_test_gru = torch.from_numpy(y_test).type(torch.Tensor)

In [237]:

input_dim = 1
hidden_dim = 32
num_layers = 2
output_dim = 1
num_epochs = 100


In [238]:

import matplotlib.pyplot as plt
import seaborn as sns

In [239]:
class GRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(GRU, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn) = self.gru(x, (h0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out


In [240]:
model = GRU(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
criterion = torch.nn.MSELoss(reduction='mean')
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)

In [241]:
def make_prediction(x_test, model, size):
    X=x_test[-1:,...,...]
    with torch.no_grad():
        predictions = []
        model.eval()
        for _ in range(size):
            yhat = model(X)
            X=torch.roll(X, shifts=-1, dims=1)
            X[0,-1,0]=yhat
            predictions.append(yhat.item())
        return torch.tensor(predictions).reshape(-1, 1)
                

In [242]:

hist = np.zeros(num_epochs)
gru = []


model = GRU(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)

# THE MODEL HAS BEEN LOADED
model = torch.load('sussy.model')
model.eval()


#loss = criterion(y_train_pred, y_train_gru)
# loss.backward()



GRU(
  (gru): GRU(1, 32, num_layers=2, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
)

In [243]:
import seaborn as sns

In [244]:
import math, time
from sklearn.metrics import mean_squared_error

# make predictions
y_test_pred = make_prediction(x_test,model,100)

# invert predictions
y_test_pred = scaler.inverse_transform(y_test_pred.detach().numpy())

In [245]:


original = scaler.inverse_transform(price['Close'].values.reshape(-1,1))

#predictions = testPredictPlot # the one of the two lines must be uncommented


result = pd.DataFrame(original)

In [246]:
p = pd.DataFrame(y_test_pred)

In [250]:
import plotly.express as px
import plotly.graph_objects as go

# predictions = np.append(trainPredictPlot, testPredictPlot, axis=1) must be uncommented for this to work

fig = go.Figure()
fig.add_trace(go.Scatter(go.Scatter(x=result.index, y=result[0],
                    mode='lines',
                    name='Original')))
fig.add_trace(go.Scatter(x=result.index+len(result[0])-1, y=p[0],
                    mode='lines',
                    name='prediction'))

fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=True,
        showticklabels=False,
        linecolor='white',
        linewidth=2
    ),
    yaxis=dict(
        title_text='Close (USD)',
        titlefont=dict(
            family='Rockwell',
            size=12,
            color='white',
        ),
        showline=True,
        showgrid=True,
        showticklabels=True,
        linecolor='white',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Rockwell',
            size=12,
            color='white',
        ),
    ),
    showlegend=True,
    template = 'plotly_dark'

)



annotations = []
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Results (GRU)',
                              font=dict(family='Rockwell',
                                        size=26,
                                        color='white'),
                              showarrow=False))
fig.update_layout(annotations=annotations)

fig.show()


In [248]:
gru = pd.DataFrame(gru, columns=['GRU'])
result

,0
0,47.58
1,47.25
2,47.65
3,47.87
4,47.08
...,...
3014,1168.36
3015,1176.76
3016,1182.26
3017,1186.10


In [249]:
result[0] #THIS IS WHAT YOU WANT. REMOVE THE FIRST THOUSAND OR SO LINES

0         47.58
1         47.25
2         47.65
3         47.87
4         47.08
         ...   
3014    1168.36
3015    1176.76
3016    1182.26
3017    1186.10
3018    1169.47
Name: 0, Length: 3019, dtype: float64